### Import library

In [ ]:
import gspread
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build

### Connect API

In [ ]:
# Đường dẫn đến file json chứa thông tin xác thực
credentials_path = 'key.json'

# Khởi tạo phiên làm việc với Google Sheets
credentials = Credentials.from_service_account_file(credentials_path, scopes=[
    'https://spreadsheets.google.com/feeds',
    'https://www.googleapis.com/auth/drive',
])
gc = gspread.authorize(credentials)

### Merge file

In [ ]:
# Google Drive folder ID chứa các Sheets gốc
old_parent_folder_id = 'drive_id_old'
new_parent_folder_id = 'drive_id_new'

# Tên của Sheet chứa dữ liệu sau khi gộp
target_sheet_name = 'dulieutonghop' #Tên gg sheet sau khi gộp

# Tạo hoặc mở một Sheet mới để chứa dữ liệu gộp
target_sheet = gc.create(target_sheet_name)

# ID của Sheet mới được tạo
target_sheet_id = target_sheet.id

# Di chuyển sheet từ thư mục cũ
drive_service = build('drive', 'v3', credentials=credentials)

# Kiểm tra xem tệp tin có tồn tại không
try:
    file_info = drive_service.files().get(fileId=target_sheet_id, fields='parents').execute()
except Exception as e:
    print(f"Lỗi khi lấy thông tin tệp tin: {e}")

# Xóa tệp tin từ thư mục cha cũ nếu nó tồn tại
if 'parents' in file_info:
    try:
        drive_service.files().update(fileId=target_sheet_id, removeParents=old_parent_folder_id).execute()
        print("The file has been successfully deleted from the old parent directory.")
    except Exception as e:
        print(f"Error deleting files from old parent directory: {e}")
else:
    print("The file does not contain the 'parents' field or does not belong to any directory, skip deletion from the old parent directory.")

# Thêm tệp tin vào thư mục cha mới
try:
    drive_service.files().update(fileId=target_sheet_id, addParents=new_parent_folder_id).execute()
    print("File added to new parent folder successfully.")
except Exception as e:
    print(f"Error adding file to new parent folder: {e}")

# Merge data from other sheets into the created sheet
sheet_names = ["s1", "s2", "s3"] #Tên các sheet

for sheet_name in sheet_names:
    source_sheet = gc.open(sheet_name).sheet1
    target_data = source_sheet.get_all_values()

    # Find or create a worksheet in the target_sheet with a name corresponding to source_sheet
    try:
        target_worksheet = target_sheet.worksheet(sheet_name)
    except gspread.exceptions.WorksheetNotFound:
        target_worksheet = target_sheet.add_worksheet(sheet_name, rows=len(target_data), cols=len(target_data[0]))

    # Update data in the target worksheet
    target_worksheet.update('A1', target_data)

print(f"The data has been merged and the worksheet is stored in the specified folder: {target_sheet.url}")